# Basic face recognizer using pre trained model



In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import sys

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

In [4]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Loading the model with pretrained weights




In [5]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [11]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [6]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str('Anurag')
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [7]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [9]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

#  run the face recognizer program :-)

In [12]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from Kunal is 0.9340575
Euclidean distance from Sumantra is 0.7224709
Euclidean distance from User_1 is 0.7080692
Euclidean distance from User_10 is 0.67442584
Euclidean distance from User_2 is 0.7318154
Euclidean distance from User_3 is 0.72745115
Euclidean distance from User_4 is 0.5870336
Euclidean distance from User_5 is 0.70585376
Euclidean distance from User_6 is 0.9235477
Euclidean distance from User_7 is 0.67803425
Euclidean distance from User_8 is 0.70256513
Euclidean distance from User_9 is 0.7416937
Euclidean distance from Kunal is 1.0532753
Euclidean distance from Sumantra is 0.64163214
Euclidean distance from User_1 is 0.46371886
Euclidean distance from User_10 is 0.64607024
Euclidean distance from User_2 is 0.59860456
Euclidean distance from User_3 is 0.6045609
Euclidean distance from User_4 is 0.49471605
Euclidean distance from User_5 is 0.549332
Euclidean distance from User_6 is 0.7472409
Euclidean distance from User_7 is 0.51464367
Euclidean distance

Euclidean distance from Kunal is 1.3981773
Euclidean distance from Sumantra is 0.8367406
Euclidean distance from User_1 is 0.7797867
Euclidean distance from User_10 is 0.8335533
Euclidean distance from User_2 is 0.84301746
Euclidean distance from User_3 is 0.8737501
Euclidean distance from User_4 is 0.86473215
Euclidean distance from User_5 is 0.7433847
Euclidean distance from User_6 is 0.7804621
Euclidean distance from User_7 is 0.72199166
Euclidean distance from User_8 is 0.974259
Euclidean distance from User_9 is 0.8919842
Euclidean distance from Kunal is 1.4499856
Euclidean distance from Sumantra is 0.8933655
Euclidean distance from User_1 is 0.81378555
Euclidean distance from User_10 is 0.915923
Euclidean distance from User_2 is 0.8901852
Euclidean distance from User_3 is 0.9312035
Euclidean distance from User_4 is 0.93418306
Euclidean distance from User_5 is 0.8176259
Euclidean distance from User_6 is 0.8372175
Euclidean distance from User_7 is 0.8132858
Euclidean distance from U

Euclidean distance from Kunal is 1.414158
Euclidean distance from Sumantra is 0.7653544
Euclidean distance from User_1 is 0.723344
Euclidean distance from User_10 is 0.8750381
Euclidean distance from User_2 is 0.7086491
Euclidean distance from User_3 is 0.76437557
Euclidean distance from User_4 is 0.7983397
Euclidean distance from User_5 is 0.68513197
Euclidean distance from User_6 is 0.60328376
Euclidean distance from User_7 is 0.6403334
Euclidean distance from User_8 is 0.99473333
Euclidean distance from User_9 is 0.9717846
Euclidean distance from Kunal is 1.3214858
Euclidean distance from Sumantra is 0.680761
Euclidean distance from User_1 is 0.598956
Euclidean distance from User_10 is 0.76795995
Euclidean distance from User_2 is 0.61099946
Euclidean distance from User_3 is 0.65611583
Euclidean distance from User_4 is 0.6505224
Euclidean distance from User_5 is 0.6113598
Euclidean distance from User_6 is 0.5259359
Euclidean distance from User_7 is 0.53887147
Euclidean distance from 

Euclidean distance from Kunal is 1.4923866
Euclidean distance from Sumantra is 0.9809723
Euclidean distance from User_1 is 0.85684854
Euclidean distance from User_10 is 1.025752
Euclidean distance from User_2 is 0.8679063
Euclidean distance from User_3 is 0.9016105
Euclidean distance from User_4 is 0.95775336
Euclidean distance from User_5 is 0.88797915
Euclidean distance from User_6 is 0.6293227
Euclidean distance from User_7 is 0.8343063
Euclidean distance from User_8 is 1.1534841
Euclidean distance from User_9 is 1.1313282
Euclidean distance from Kunal is 1.4899523
Euclidean distance from Sumantra is 0.98204535
Euclidean distance from User_1 is 0.831559
Euclidean distance from User_10 is 1.020004
Euclidean distance from User_2 is 0.8402573
Euclidean distance from User_3 is 0.8705557
Euclidean distance from User_4 is 0.92662126
Euclidean distance from User_5 is 0.84057575
Euclidean distance from User_6 is 0.62000495
Euclidean distance from User_7 is 0.7902807
Euclidean distance from 

Euclidean distance from Kunal is 1.2343743
Euclidean distance from Sumantra is 0.51081544
Euclidean distance from User_1 is 0.59586215
Euclidean distance from User_10 is 0.73411185
Euclidean distance from User_2 is 0.6299263
Euclidean distance from User_3 is 0.65819526
Euclidean distance from User_4 is 0.57087874
Euclidean distance from User_5 is 0.58418447
Euclidean distance from User_6 is 0.68123937
Euclidean distance from User_7 is 0.56987935
Euclidean distance from User_8 is 0.8667181
Euclidean distance from User_9 is 0.8026387
Euclidean distance from Kunal is 1.2136605
Euclidean distance from Sumantra is 0.53129286
Euclidean distance from User_1 is 0.650077
Euclidean distance from User_10 is 0.7587307
Euclidean distance from User_2 is 0.6646593
Euclidean distance from User_3 is 0.65721065
Euclidean distance from User_4 is 0.55247617
Euclidean distance from User_5 is 0.636635
Euclidean distance from User_6 is 0.76737005
Euclidean distance from User_7 is 0.5978156
Euclidean distance

...

Euclidean distance from Kunal is 1.2906559
Euclidean distance from Sumantra is 0.8871818
Euclidean distance from User_1 is 0.6537317
Euclidean distance from User_10 is 0.9646651
Euclidean distance from User_2 is 0.75275177
Euclidean distance from User_3 is 0.7059505
Euclidean distance from User_4 is 0.78023773
Euclidean distance from User_5 is 0.76929647
Euclidean distance from User_6 is 0.6854288
Euclidean distance from User_7 is 0.748279
Euclidean distance from User_8 is 1.006034
Euclidean distance from User_9 is 1.0609257
Euclidean distance from Kunal is 1.3428026
Euclidean distance from Sumantra is 1.0608581
Euclidean distance from User_1 is 0.75977296
Euclidean distance from User_10 is 1.082275
Euclidean distance from User_2 is 0.8997608
Euclidean distance from User_3 is 0.87347394
Euclidean distance from User_4 is 0.94044846
Euclidean distance from User_5 is 0.9310506
Euclidean distance from User_6 is 0.8315537
Euclidean distance from User_7 is 0.918251
Euclidean distance from Us

Euclidean distance from Kunal is 1.4575207
Euclidean distance from Sumantra is 0.9764755
Euclidean distance from User_1 is 0.7856527
Euclidean distance from User_10 is 0.9190541
Euclidean distance from User_2 is 0.7973991
Euclidean distance from User_3 is 0.80862397
Euclidean distance from User_4 is 0.87369174
Euclidean distance from User_5 is 0.7640556
Euclidean distance from User_6 is 0.5947701
Euclidean distance from User_7 is 0.6542039
Euclidean distance from User_8 is 1.0270692
Euclidean distance from User_9 is 1.0199249
Euclidean distance from Kunal is 1.2603798
Euclidean distance from Sumantra is 0.73736227
Euclidean distance from User_1 is 0.51867837
Euclidean distance from User_10 is 0.6576375
Euclidean distance from User_2 is 0.59410787
Euclidean distance from User_3 is 0.6146575
Euclidean distance from User_4 is 0.6182222
Euclidean distance from User_5 is 0.47097057
Euclidean distance from User_6 is 0.5907301
Euclidean distance from User_7 is 0.4101189
Euclidean distance fro

Euclidean distance from Kunal is 1.3987577
Euclidean distance from Sumantra is 0.8792762
Euclidean distance from User_1 is 0.65471274
Euclidean distance from User_10 is 0.8780287
Euclidean distance from User_2 is 0.7053931
Euclidean distance from User_3 is 0.7100361
Euclidean distance from User_4 is 0.7576094
Euclidean distance from User_5 is 0.69278294
Euclidean distance from User_6 is 0.5569794
Euclidean distance from User_7 is 0.58555675
Euclidean distance from User_8 is 0.98592013
Euclidean distance from User_9 is 0.9858362
Euclidean distance from Kunal is 1.3533076
Euclidean distance from Sumantra is 0.81967604
Euclidean distance from User_1 is 0.60473585
Euclidean distance from User_10 is 0.79860336
Euclidean distance from User_2 is 0.6913925
Euclidean distance from User_3 is 0.7004345
Euclidean distance from User_4 is 0.72104913
Euclidean distance from User_5 is 0.6514565
Euclidean distance from User_6 is 0.62404037
Euclidean distance from User_7 is 0.5516529
Euclidean distance 

Euclidean distance from Kunal is 1.3869107
Euclidean distance from Sumantra is 0.8184376
Euclidean distance from User_1 is 0.64460886
Euclidean distance from User_10 is 0.8515624
Euclidean distance from User_2 is 0.6983005
Euclidean distance from User_3 is 0.7010958
Euclidean distance from User_4 is 0.7206193
Euclidean distance from User_5 is 0.6638926
Euclidean distance from User_6 is 0.5698535
Euclidean distance from User_7 is 0.54120505
Euclidean distance from User_8 is 0.967987
Euclidean distance from User_9 is 0.961711
Euclidean distance from Kunal is 1.3785063
Euclidean distance from Sumantra is 0.8144652
Euclidean distance from User_1 is 0.6440402
Euclidean distance from User_10 is 0.84394115
Euclidean distance from User_2 is 0.7392965
Euclidean distance from User_3 is 0.7429847
Euclidean distance from User_4 is 0.7654749
Euclidean distance from User_5 is 0.6798986
Euclidean distance from User_6 is 0.64378107
Euclidean distance from User_7 is 0.5700373
Euclidean distance from Us

Euclidean distance from Kunal is 1.0158145
Euclidean distance from Sumantra is 1.0309799
Euclidean distance from User_1 is 1.1045561
Euclidean distance from User_10 is 1.0011632
Euclidean distance from User_2 is 1.0582074
Euclidean distance from User_3 is 1.0827765
Euclidean distance from User_4 is 1.0694106
Euclidean distance from User_5 is 1.042885
Euclidean distance from User_6 is 1.2196617
Euclidean distance from User_7 is 1.1437851
Euclidean distance from User_8 is 0.91302866
Euclidean distance from User_9 is 1.0381513
Euclidean distance from Kunal is 1.0594653
Euclidean distance from Sumantra is 0.9444975
Euclidean distance from User_1 is 0.9038637
Euclidean distance from User_10 is 0.8901226
Euclidean distance from User_2 is 1.027944
Euclidean distance from User_3 is 1.0441735
Euclidean distance from User_4 is 0.96752226
Euclidean distance from User_5 is 0.94771326
Euclidean distance from User_6 is 1.1004217
Euclidean distance from User_7 is 0.92880124
Euclidean distance from Us